# Исследование надежности заемщиков.

*Поступил запрос от заказчика. Ему требуется ответ на вопрос: "Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок?" В данном проекте стоит разобраться, действительно ли семейное положение влияет на погашение кредита в срок, а также существуют ли какие-либо ещё взаимосвязи между погашением кредита и данными кредитора/типом кредита.*

*Первым делом нам стоит посмотреть на данные, которые имеются у нас в наличии. Какого они типа? Имеются ли в них пропущенные значения? Для этого мы должны обратиться к датасету. Приступим ;)*

## Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


*Как мы видим, в столбцах "days_employed" и "total_income" имеются пропущенные значения. А также, только эти два столбца имеют тип float. Взглянем на них, чтобы определить как выглядят пропущенные занчения. Для этого проверим все строки в столбцах и оставим только те, в которых есть NaN. А затем выведем первые 5 из них*

In [3]:
data[data['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


*На первый взгялд, пропущенные значения обозначаются, как NaN.* 

*Посчитаем, какую долю составлют пропущенные значения для каждого из столбцов.*

In [33]:

data.isna().mean()

children                 0.0
days_employed            0.0
dob_years                0.0
education                0.0
education_id             0.0
family_status            0.0
family_status_id         0.0
gender                   0.0
income_type              0.0
debt                     0.0
total_income             0.0
purpose                  0.0
total_income_category    0.0
purpose_category         0.0
dtype: float64

*Количество пропущенных значений составляет меньше 1%.* 
*У данных пропусков есть несколько возможных причин:*
- Если они заполняются вручную, то человек мог просто не указать нужные данные о себе.
- При выгрузке данных, часть из них была утеряна.

## Шаг 2 Заполнение пропусков

*Мы выяснили достаточно. Начнём заполнение пропущенных значений со столбца "total_income". Заполним данные в нём медианными значениями.*

*Данный тип заполнения подходит больше всего, поскольку данные в этом столбце отражают ежемесячных доход, то некорректно было бы заполнять его средним арифметическим, поскольку тогда данные были бы не совсем верные. Ведь если в выборке присутствуют выдающиеся значения (прим. 1000 против 10), лучше использовать медиану, во избежания искажения данных.*

*Заполним пропуски медианным значением в зависимости от типа занятости, а затем проверим, остались ли они.*

In [7]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform("median"))
data[data['total_income'].isna()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

*Ура! Все пропущенные строки заполнились медианными значениями. Можем переходить к следующему столбцу "days_empoyed"*

## Шаг 3 Проверка данных на аномалии и исправления.

*Также мы знаем, что в столбце "days_employed" могут присутствовать аномалии, а если они обнаружатся, значит нужно с ними разобраться. Попытаемся найти эти аномалии и проанализируем их.*

*Первым делом попробуем найти отрицательные значения. Для этого посчитаем количество строк с отрицательными и положительными значениями.*

In [8]:
positive_lines = 0
negative_lines = 0
 
for days in data['days_employed']:
    if days > 0:
        positive_lines += 1
    else:
        negative_lines += 1
print(positive_lines)
print(negative_lines)

3445
18080


*При первой проверке в данных более 80% данных оказались с отрицательными значениями. Выведем некоторые из них на экран.*

In [9]:
display(data[data['days_employed'] < 0].head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья


*Действительно, в нашем столбце 84% данных перепутались и стали иметь отрицательное значение. Вернем им положительное значение, чтобы сделать данные корректными.*

In [10]:
data['days_employed'] = data['days_employed'].abs()

*Проверим, остались ли отрицательные значения.*

In [11]:
data[data['days_employed'] < 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

*Отлично, теперь можем заполнить пропуски корректными медианными значениями. Сделаем это и проверим, остались ли они.*

In [12]:
data['days_employed'] = data['days_employed'].fillna(value = data['days_employed'].median())
data[data['days_employed'].isna()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

*Теперь проверим наличие аномалий в столбце "children". Для этого выведем уникальные значения из этого столбца. А затем проверим, сколько раз встречается аномальное значение.*

In [13]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [14]:
data.loc[data['children'] == -1, ['children']].count()

children    47
dtype: int64

*Как мы видим, здесь есть значение -1, а такое вряд ли возможно. Поскольку такое значение встречается всего у 47 заёмщиков(что меньше одного процента), то принято решение удалить строки с данными аномальными значениями, поскольку их отсутствие не сильно исказит финальный подсчёт.*

In [15]:
data = data.loc[data['children'] >= 0]

In [16]:
data.loc[data['children'] < 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

*Отлично! Мы закончили работу со всеми аномалиями. Но стоит задуматься от чего же мог появится такой большой процент в dats_employed?*
*Есть несколько возможных вариантов:*
* *Ошибка произошла при выгрузке данных. Они повредились и что-то пошло не так.*
* *Возможно данные были неверно закодированны*
 
*Но можно точно сказать, что это не случайная опечатка, вызванная человеческим фактором.*

## Шаг 4 Изменение типов данных.

*Теперь займёмся изменением типа данных. Мы уже видели, что в начале в датасете столбцы "days_employed", "total_income" и "children" имеют тип float. Поэтому изменим данный тип данных на int.*

In [17]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int') 
data['children'] = data['children'].astype('int') 

*После изменения проверим типы данных.*

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21478 non-null  int64 
 1   days_employed     21478 non-null  int64 
 2   dob_years         21478 non-null  int64 
 3   education         21478 non-null  object
 4   education_id      21478 non-null  int64 
 5   family_status     21478 non-null  object
 6   family_status_id  21478 non-null  int64 
 7   gender            21478 non-null  object
 8   income_type       21478 non-null  object
 9   debt              21478 non-null  int64 
 10  total_income      21478 non-null  int64 
 11  purpose           21478 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


## Шаг 5 Удаление дубликатов.

*Следующий шаг в обработке наших данных - проверка на дубликаты. Сначала посчитаем, а затем выведем на экран яные дубликаты.*

In [19]:
data.duplicated().sum()

54

In [20]:
data[data.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
4182,1,2194,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,2194,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,2194,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
7808,0,2194,57,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
8583,0,2194,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,145017,дополнительное образование
9238,2,2194,34,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для сдачи
9528,0,2194,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145017,операции со своей недвижимостью
9627,0,2194,56,среднее,1,женат / замужем,0,F,пенсионер,0,145017,операции со своей недвижимостью
10462,0,2194,62,среднее,1,женат / замужем,0,F,пенсионер,0,145017,покупка коммерческой недвижимости


*На первый взгляд тяжело сказать, почему возникают такого рода дубликаты. Возможно ошибка кроется в заполнении данных, а может быть один и тот же человек брал кредит несколько раз, в следствие чего был внесён в базу дважды.*

*Удалим явные дубликаты известным нам методом и проверим остались ли они.*

In [21]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

*Отлично, от явлых дубликатов мы избавились. Далее искать неявные дубликаты будем в тех столбцах, которые имеют категориальное значение. Это столбцы "education", "family_status", "gender", "income_type", "purpose". Выведем уникальные занчения этих столбцов на экран и поищем дубликаты среди них.*

In [22]:
print(data['education'].unique())
print(data['family_status'].unique())
print(data['gender'].unique())
print(data['income_type'].unique())
print(data['purpose'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['F' 'M' 'XNA']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образо

*Как мы видим, в столбце "education" есть дубликаты, которые связаны с регистром. Остальные же столбцы не имеют дубликатов (кроме "purpose", но с категоризацией целей кредита мы разберёмся позже, пока отчистим "education" от дубликатов.)*

In [23]:
data['lower_education'] = data['education'].str.lower()
data['education'] = data['lower_education']
data.pop('lower_education')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


*Отлично, проверим все ли дубликаты мы удалили.*

In [24]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

*Отлично, мы отчистили данный столбец от думбликатов, пора переходить к следующей задаче: формированию дополнительных датафреймов словарей и декомпозицией исходного датафрейма.*

## Шаг 6 Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

*Для удобства мы создадим дополнительный датафрейм, который будет хранить в себе идентификатор и данные о двух столбцах: 'education' и 'family_status'. А затем отчистим исходный датафрейм от данных столбцов, оставив только их идентификатор.*

In [25]:
education_dict = pd.DataFrame(data.loc[:, 'education':'education_id'])
family_dict = pd.DataFrame(data.loc[:, 'family_status':'family_status_id'])
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
data.drop(['education', 'family_status'], axis=1)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


## Шаг 7 Категоризация дохода.

*Для того, чтобы человек, которому нужно понять категорию заёмщика, мог быстро это сделать, нам нужно категоризировать данные по доходу. Составим дополнительных столбец, куда внесём категории заёмщиков, распределив их по доходу из столбца "total_income", а затем выведем первые 5 строк датафрейма для проверки.*

In [26]:
def category(total_income): 
    try:
        if 0 < total_income < 30000:
            return 'E'
        elif 30001 < total_income < 50000:
            return 'D'
        elif 50001 < total_income < 200000:
            return 'C'
        elif 200001 < total_income < 1000000:
            return 'B'
        else:
            return 'A'
    except:
        return 'нет категории'
data['total_income_category'] = data['total_income'].apply(category)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


*Отлично, теперь сразу понятно, кто к какой категории относится. Теперь проделаем тоже самое с целями кредита.*

## Шаг 8 Категоризация целей кредита.

*Теперь нам нужно разбить цели кредита на отдельные категории. Для начала вновь выведем уникальные значения, ориентируясь на которые, мы будем категоризировать цели кредита.*

In [27]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

*Выводя уникальные значения, мы обнаружили множество повторов в целях кредита, которые отличаются лишь формулировкой и/или регистром. Нужно это исправить. Создадим дополнительный столбец, в котором будут отражены цели кредита, распределённые на 4 категории: "операции с автомобилем", "операции с недвижимостью", "проведение свадьбы", "получение образования".*

In [28]:
def credit_purpose(purpose):
    try:
        if 'авто' in purpose:
            return 'операции с автомобилем'
        if 'недвиж' in purpose or 'жил' in purpose:
            return 'операции с недвижимостью'
        if 'свад' in purpose:
            return 'проведение свадьбы'
        if 'образов' in purpose:
            return 'получение образования'
    except:
        return 'нет категории'
data['purpose_category'] = data['purpose'].apply(credit_purpose)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


*Отлично, цели кредита чётко известны в отдельном столбце. Данные обработаны и готовы к анализу! Пришло время проверить гипотезы.*

## Ответы на вопросы.

### *Вопрос 1:* 
*Есть ли зависимость между количеством детей и возвратом кредита в срок?*

In [29]:
children_result = data.pivot_table(index = 'children', columns = 'debt', values = 'days_employed', aggfunc ='count')
children_result['result'] = children_result[1]/(children_result[0] + children_result[1]) * 100
children_result

debt,0,1,result
children,,,
0,13044.0,1063.0,7.535266
1,4365.0,444.0,9.232689
2,1858.0,194.0,9.454191
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN
20,68.0,8.0,10.526316


##### *Вывод 1:*
*Из полученной таблицы можно сделать следующий вывод: люди, у которых есть как минимум 1 ребёнок, больше склонны к тому, чтобы не вернуть кредит в срок, чем те, у кого детей нет. Больше всего просрочек у людей, которые имеют 2, 4 и 20 детей.*

### *Вопрос 2:*
*Есть ли зависимость между семейным положением и возвратом кредита в срок?*

In [30]:
family_status_result = data.pivot_table(index = 'family_status_id', columns = 'debt', values = 'days_employed', aggfunc ='count')
family_status_result['result'] = family_status_result[1]/(family_status_result[0] + family_status_result[1]) * 100
family_status_result.reset_index().merge(family_dict)

,family_status_id,0,1,result,family_status
0,0,11385,930,7.551766,женат / замужем
1,1,3770,388,9.331409,гражданский брак
2,2,892,63,6.596859,вдовец / вдова
3,3,1106,85,7.136860,в разводе
4,4,2531,274,9.768271,Не женат / не замужем


#### *Вывод 2:*
*Из полученной таблицы можно сделать следующий вывод: больше всего склонны к невозврату кредита в срок люди, которые состоят в гражданском браке или не состоят в нём вообще. Процент недобросовестных заёмщиков в этих категориях равен 9%.*

### *Вопрос 3:* 
*Есть ли зависимость между уровнем дохода и возвратом кредита в срок?*

In [31]:
total_income_result = data.pivot_table(index = 'total_income_category', columns = 'debt', values = 'days_employed', aggfunc ='count')
total_income_result['result'] = total_income_result[1]/(total_income_result[0] + total_income_result[1])  * 100
total_income_result

debt,0,1,result
total_income_category,,,
A,24,2,7.692308
B,4676,356,7.074722
C,14636,1359,8.496405
D,328,21,6.017192
E,20,2,9.090909


#### *Вывод 3:*
*Из полученной таблицы можно сделать следующий вывод: больше всего недобросовестных заёмщиков среди категории С. То есть, люди со средним доходом больше склонны не возвращать кредит в срок, нежели люди с более высоким или низким уровнем доходов. (По процентам также лидирует категория Е, но говорить о том, что там больше всего уровень просрочки будет некорректно, так как сильно отличается количество человек, анализируемые в данном вопросе).*

### *Вопрос 4:*
*Как разные цели кредита влияют на его возврат в срок?*

In [32]:
purpose_category_result = data.pivot_table(index = 'purpose_category', columns = 'debt', values = 'days_employed', aggfunc ='count')
purpose_category_result['result'] = purpose_category_result[1]/(purpose_category_result[0] + purpose_category_result[1]) * 100
purpose_category_result

debt,0,1,result
purpose_category,,,
операции с автомобилем,3895,402,9.355364
операции с недвижимостью,10008,782,7.247451
получение образования,3634,370,9.240759
проведение свадьбы,2147,186,7.972568


#### *Вывод 4:*
*Из выведенной таблицы можно сделать следующий вывод: больше всего склонны не возвращать кредит в срок те люди, которые взяли его на операции с автомобилем и получением образования. Средти таких заёмщиков уровень невозвратов равно 9%.*

## Общий вывод:

*В данном проекте мы исследовали то, какие именно переменные влияют на возврат кредита в срок. Благодаря анализу удалось выявить, что как минимум три фактора влияют на возврат кредита в срок, а именно: цель кредита, семейное положение и доход заёмщика. Как вы выяснили, портрет заёмщика, который с большей долей вероятности просрочит кредит, будет в большинстве случаев выглядеть так: это человек со средним уровнем дохода(50 - 200 тыс. руб), который имеет как минимум одного ребёнка(как правило 2ух/4ех или 20), который состоит в гражданском браке(или вообще не состоит в браке), а также целью его кредита будут операции с автомобилем, либо же получение образования. Эти данные можно будет использовать при оценке будущего заёмщика с целью просчитать заранее, каковы шансы того, что долг будет выплачен вовремя.*